# Visitor: reflective visitor

> Improving on the invasive visitor

In the previous lesson we decided to implement printing by adding a `print` method to both of our classes, thereby violating the Open-Closed Principle. Let's remove the method and implement a separate class instead (we will ignore evaluation for now).

In [ ]:
class DoubleExpression:
    def __init__(self, value):
        self.value = value

class AdditionExpression:
    def __init__(self, left, right):
        self.right = right
        self.left = left
        
class ExpressionPrinter:
    """Our new class for printing"""
    @staticmethod
    def print(e, buffer):
        if isinstance(e, DoubleExpression):
            buffer.append(str(e.value))
        elif isinstance(e, AdditionExpression):
            buffer.append('(')
            ExpressionPrinter.print(e.left, buffer)
            buffer.append('+')
            ExpressionPrinter.print(e.right, buffer)
            buffer.append(')')

`ExpressionPrinter.print` takes 2 arguments: an expression `e` and a `buffer`. Inside the method we need to check the kind of expression we receive in order to know what to add to the buffer; for the specific case of `AdditionExpression`, we add the brackets and the plus sign and recursively call the method for each side.

Let's use our new class.

In [2]:
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
buffer = []

ExpressionPrinter.print(e, buffer)
print(''.join(buffer))

(1+(2+3))


Excellent, we get the same result as before (minus the evaluation, which we are ignoring for now). We now have all of the printing stuff in a separate class and we don't need to modify our expression classes for anything.

However, our expression `e` is now missing the `print` method, so we cannot use `e.print(buffer)` anymore, but we can give back the `print` method to all expressions with inheritance: we will create a new `Expression` base class that all other expression classes can inherit from, and then we'll add a new line to the `ExpressionPrinter` class.

In [3]:
from abc import ABC

class Expression(ABC):
    """Our new and empty base class"""
    pass

class DoubleExpression(Expression): # we inherit from Expression now
    def __init__(self, value):
        self.value = value

class AdditionExpression(Expression): # we inherit from Expression now
    def __init__(self, left, right):
        self.right = right
        self.left = left
        
class ExpressionPrinter:
    @staticmethod
    def print(e, buffer):
        if isinstance(e, DoubleExpression):
            buffer.append(str(e.value))
        elif isinstance(e, AdditionExpression):
            buffer.append('(')
            ExpressionPrinter.print(e.left, buffer)
            buffer.append('+')
            ExpressionPrinter.print(e.right, buffer)
            buffer.append(')')

    # Here's where the magic happens
    Expression.print = lambda self, b: ExpressionPrinter.print(self, b)

The last line inside `ExpressionPrinter` is assigning a new `print` variable to `Expression` which contains a lambda function that invokes `ExpressionPrinter.print` with both the `Expression` instance and the buffer. This makes it possible for us to call `e.print(buffer)` like before without having to explicitly modify any of the classes:

In [4]:
# represents 1+(2+3)
e = AdditionExpression(
    DoubleExpression(1),
    AdditionExpression(
        DoubleExpression(2),
        DoubleExpression(3)
    )
)
buffer = []

# IDE might complain here
e.print(buffer)

print(''.join(buffer))

(1+(2+3))


The downside to this approach is that, should you want to add new functionality such as a `SubtractionExpression` class, the `print` methods needs to be updated each time to take the new class into consideration, because otherwise it would be ignored.

> Note: in the original course, the teacher refers to `ExpressionPrinter.print` as a "reflective printer", because "in some programming languages, the checking of the type [referring to `isinstance` inside `print`] is a reflective operation". However, the `Expression.print` assignment from within `ExpressionPrinter` seems to be a form of [monkey-patchin](https://stackoverflow.com/questions/5626193/what-is-monkey-patching); the teacher does not mention this at all in the course.